# Práctica 4

**Nombre:** Gutierrez Ramirez Felipe de Jesus

**e-mail:** felipe.gutierrez5025@alumnos.udg.mx

## MODULES

In [1]:
import math
import numpy as np
import pandas as pd
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

import plotly.graph_objects as go
import panel as pn
import panel.widgets as pnw
pn.extension('plotly')

## CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [3]:
######################################################################
# Brownian Motion Trajectory
######################################################################

def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
    n_steps:
    speed:
    s_pos:
  Returns:
    BM_2d_df:
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

In [4]:
######################################################################
# Correlated Random Walk Trajectory
######################################################################

def crw(CRW_exponent=0.4, n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
    CRW_exponent:
    n_steps:
    speed:
    s_pos:
  Returns:
    CRW_df:
  """
  velocity = Vec2d(speed,0)
  pos = Vec2d(s_pos)
  cauchy_samples = wrapcauchy.rvs(CRW_exponent, size=n_steps, loc=0)

  cauchy = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
  cauchy = pd.concat([cauchy,temp_df], ignore_index=True)

  for i in range(n_steps-1):
    # Elegir el angulo de giro
    velocity = velocity.rotated(cauchy_samples[i])
    pos += velocity
    temp_df = pd.DataFrame([{'x_pos': pos.x, 'y_pos': pos.y}])
    cauchy = pd.concat([cauchy,temp_df], ignore_index=True)

  return cauchy

In [5]:
######################################################################
# Levy flight Trajectory
######################################################################
def levy(alpha=0.9, beta=1.0, loc=0,CRW_exponent=0.5, n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
  alpha:
  beta:
  loc:
  CRW_exponent:
  n_steps:
  speed:
  s_pos:
  Returns:
  Levy_df:
  """
  velocity = Vec2d(speed,0)
  levy = [Vec2d(s_pos[0], s_pos[1])]
  angles = wrapcauchy.rvs(c=CRW_exponent, size=n_steps)
  step_lengths = levy_stable.rvs(alpha, beta, loc, scale=speed,size=n_steps)
  for i in range(n_steps):
    velocity = velocity.rotated(angles[i])
    steps = Vec2d(velocity.x * step_lengths[i], velocity.y * step_lengths[i])
    new_pos = levy[-1] + steps
    levy.append(new_pos)
  
  levy_df = pd.DataFrame([(step.x, step.y) for step in levy], columns=['x_pos', 'y_pos'])
  return levy_df

In [6]:
# Define your function to compute path length for given trajectory

## start - Add your code here
def path_length(traj):
    lengths = pd.DataFrame(columns=['distance'])
    temp_df = pd.DataFrame([{'distance': 0}])
    lengths =pd.concat([lengths,temp_df], ignore_index=True)
    for i in range(1, len(traj)):
        x0, y0 = traj.iloc[i - 1]['x_pos'], traj.iloc[i - 1]['y_pos']
        x1, y1 = traj.iloc[i]['x_pos'], traj.iloc[i]['y_pos']
        length = lengths.iloc[-1]['distance'] + np.sqrt((x1 - x0)**2 + (y1 - y0)**2)
        temp_df = pd.DataFrame([{'distance': length}])
        lengths =pd.concat([lengths,temp_df], ignore_index=True)
    return lengths

## end - Add your code here

In [7]:
# Define your function to compute Mean Squared Displacement for given trajectory

## start - Add your code here
def msd(traj):
    N = len(traj)
    msd_list = []
    for i in range(1, N):
        squared_displacements = (traj.iloc[i:] - traj.iloc[:-i].values) ** 2
        msd = squared_displacements.sum(axis=1).mean()
        msd_list.append(msd)
    msd = pd.DataFrame({'MSD': msd_list})
    return msd

## end - Add your code here

## Widgets

In [8]:
#Selectores
trajectories = pnw.RadioButtonGroup(name='Trayectorias', options=['Brownian', 'CRW', 'Levy'], margin=(0,0,0,0), width=200)
metrics = pnw.Select(name='Metricas', options=['MSD', 'Path Length'], width=200)
#Parametros
n_steps = pnw.IntInput(name='Numero de pasos', start=100, end=100000,step=100, width=200)
speed = pnw.IntInput(name='Velocidad', start=1,end=10, step=1, width=200)
x_initial = pnw.IntInput(name='X inicial', start=0, end=10, step=1, width=200)
y_initial = pnw.IntInput(name='Y inicial', start=0, end=10, step=1, width=200)
initial_pos = pn.Column(x_initial,y_initial)
crw_exponent = pnw.FloatSlider(name='CRW Exponent', start=0.5, end=1.0, step=0.1, width=200)
alpha = pnw.FloatSlider(name='Alpha', start=0.9, end=2.0, step=0.1, width=200)
# Graficas
trajectory_plot = pn.pane.Plotly()
metric_plot = pn.pane.Plotly()

dynamic_widgets = pn.Column()

## Funciones para Panel

In [9]:
#Widgets dinamicos
def update_widgets(trajectory):
  if trajectory == 'Brownian':
    return []
  elif trajectory == 'CRW':
    return [crw_exponent]
  elif trajectory == 'Levy':
    return [crw_exponent,alpha]
  
#Cambio de trayectoria
@pn.depends(trajectories.param.value)
def parameters(trajectory):
  dynamic_widgets.clear()
  widgets = update_widgets(trajectory)
  if widgets:
      dynamic_widgets.extend(widgets)
  return dynamic_widgets

## Lanzar panel

In [15]:
pn.config.sizing_mode = 'stretch_width'  # Para limitar la expansión horizontal

# Crea tu dashboard aquí
dashboard = pn.Column(
    pn.Row(trajectories,metrics),
    n_steps,
    speed,
    initial_pos,
    parameters,
    pn.Row(trajectory_plot, metric_plot)
)

pn.config.max_width = 300  # Establece el ancho máximo
pn.config.max_height = 300  # Establece la altura máxima

dashboard.servable(title='Dashboard')

BokehModel(combine_events=True, render_bundle={'docs_json': {'99bef335-4919-4cf7-99f5-de6feff799dd': {'version…